# Task 1: timing and profiling matrix multiplication

To start out we are going to take some of the lessons learned in the timing and profiling lesson and apply them to a simple problem: multiplying two matricies together.

We begin by creating some test data using python built-ins. Python has a `list` type, but it's just a collection of objects. You can emulate a 2d array or matrix by creating a list-of-lists like this:

In [ ]:
mat = [[0,1,2],
       [3,4,5],
       [6,7,8]]

We want to test our functions using some large matrix problems so we'll use numpy to generate a 50x50 array of integers between 0 and 10.

Note that we convert these numpy arrays into python list-of-lists at the end.

In [ ]:
import numpy as np

In [ ]:
# use numpy to generate some large random integer arrays
np.random.seed(12345)
shape = (50,50)
A = np.random.randint(0, high=10, size=shape)
B = np.random.randint(0, high=10, size=shape)
# Compute the answer for future testing
C = np.matmul(A,B)
# Conver to python list-of-list
A = A.tolist()
B = B.tolist()
C = C.tolist()

In [ ]:
type(A)

Now that we have test data we can write our matrix multiplication function.

Our goal is to have $C_{ij} = \Sigma_k (A_{ik}*B_{kj})$ which means we need a 3x nested loop.

We also need to check that the input data are actually formed correctly and that they are compatible for matrix multiplication.

In [ ]:
def mat_mul_0(A,B):
    """
    Compute the matrix multiplicatoin of A and B.
    Assuming that A,B are python lists of lists and we use only pure python.
    
    Adapted from https://stackoverflow.com/a/10508133/1710603
    """
    rows_A = len(A)
    cols_A = len(A[0])
    rows_B = len(B)
    cols_B = len(B[0])

    if cols_A != rows_B:
      return None

    # create a results matrix initialised to zeros
    C = [[0 for row in range(cols_B)] for col in range(rows_A)]

    # recall this is actually a triple loop, once each for the dimensions of C, and again for the sum over k
    # C_ij = SUM_k ( A_ik*B_kj)
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                C[i][j] += A[i][k] * B[k][j]
    return C

It is good practice when profiling to ensure that your optimisations don't break your code. A solid test suite will come in hand here. For now we just write a simple function as our test

In [ ]:
def correct(A, B, answer, func):
    """
    Test that AxB == answer
    """
    return np.all(func(A,B)==answer)

In [ ]:
correct(A,B,C,mat_mul_0)

Once we confirm that the function gives the right result, we can time it to get a baseline for future comparison.

In [ ]:
%timeit mat_mul_0(A,B)

Now let us try our hand at some optimisation.

The line `C[i][j] += A[i][k] * B[k][j]` is making repeated read/write access to an array element in C. This involves two lookups, one for each index.
This sounds like a bad idea so our first test is to see if we can refactor the code to use a more local variable to store the data during the loop and then write to the C matrix just once.

In [ ]:
def mat_mul_1(A,B):
    """
    Compute the matrix multiplicatoin of A and B.
    with temp storage
    """
    rows_A = len(A)
    cols_A = len(A[0])
    rows_B = len(B)
    cols_B = len(B[0])

    if cols_A != rows_B:
      return None

    # create a results matrix initialised to zeros
    C = [[0 for row in range(cols_B)] for col in range(rows_A)]

    # recall this is actually a triple loop, once each for the dimensions of C, and again for the sum over k
    # C_ij = SUM_k ( A_ik*B_kj)
    for i in range(rows_A):
        for j in range(cols_B):
            n =0
            for k in range(cols_A):
                n += A[i][k] * B[k][j]
            C[i][j] =n
    return C

In [ ]:
correct(A,B,C,mat_mul_1)

In [ ]:
%timeit mat_mul_1(A,B)

Did that help?

Suppose that we insted wanted to do away with the temp variable, and collapse one of our loops using a generator function.

That is, we create a generator which returns the `A[i][k] * B[k][j]` result for each `k` and then we `sum()` over that.
This is called list comprehension and it is a common way to in-line or one-line a simple loop.

Lets see if it will make a difference to our run time.

In [ ]:
def mat_mul_2(A,B):
    """
    Compute the matrix multiplicatoin of A and B.
    with list comprehension
    """
    rows_A = len(A)
    cols_A = len(A[0])
    rows_B = len(B)
    cols_B = len(B[0])

    if cols_A != rows_B:
      return None

    # create a results matrix initialised to zeros
    C = [[0 for row in range(cols_B)] for col in range(rows_A)]

    # recall this is actually a triple loop, once each for the dimensions of C, and again for the sum over k
    # C_ij = SUM_k ( A_ik*B_kj)
    for i in range(rows_A):
        for j in range(cols_B):
            C[i][j] = sum(A[i][k] * B[k][j] for k in range(cols_A))
    return C

In [ ]:
correct(A,B,C, mat_mul_2)

In [ ]:
%timeit mat_mul_2(A,B)

How did that work out?

Lets go crazy and replace **all** the loops with list comprehension.

Bonus, we don't have to intialise the C matrix!

This should be amazing!

In [ ]:
def mat_mul_3(A,B):
    """
    Compute the matrix multiplicatoin of A and B.
    with full list comprehension
    """
    rows_A = len(A)
    cols_A = len(A[0])
    rows_B = len(B)
    cols_B = len(B[0])

    if cols_A != rows_B:
      return None

    C = [ [sum(A[i][k] * B[k][j] for k in range(cols_A)) for j in range(cols_B)] for i in range(rows_A)]
    return C

In [ ]:
correct(A,B,C,mat_mul_3)

In [ ]:
%timeit mat_mul_3(A,B)

This isn't going as planned.

So far we have mostly just been working on hunches and guesses as to what is causing the various slow downs.
Lets do a proper profile of this funciton to see where the slow points really are.

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f mat_mul_2 mat_mul_2(A,B)

In [ ]:
%lprun -f mat_mul_3 mat_mul_3(A,B)

## Conclusion 1: Python loops are already fast
List comprehensions are just python loops. Python loops are already extremely well optimised (in python 3) so there is no (speed) optimisation to be had.
List comprehensions tend to be harder to read, and harder to profile, especially when they are chained together like this.

Use list comprehensions sparingly.

In fact, for operations such as matrix multiplication we should not rely on pure python code.
Instead we should look for libraries that do this manipulation for us.

The numerical python library `numpy` has a matrix multiplication method called `matmul` that will do this for us.
As a bonus, this relies on C-level library calls, which use optimised linear algebra libraries on your system like BLAS, LAPAC, ATLAS, etc (if installed)

In [ ]:
# create another function which is just the numpy version
mat_mul_4 = np.matmul

In [ ]:
#Test that we get the right result
correct(A,B,C,mat_mul_4)

In [ ]:
%timeit mat_mul_4(A,B)

Notice that this now takes **micro**seconds instead of **milli**seconds to run. Seems like we'll get the best result just by implementing existing code.

One caveat is that `numpy` works with it's own data types so in the call above we are converting the A,B list-of-list into a numpy.ndarray before doing the multiplication
If we instead use numpy data types directly we get a different result:

In [ ]:
D = np.array(A)
E = np.array(B)

In [ ]:
%timeit mat_mul_4(D,E)

The overhead in converting a python list into a numpy ndarray is *significant*, and in this case it is **greater than the time to compute our result**.

## Conclusion 2: Don't re-invent the wheel
Look for an use existing libraries before you write code, and **especially** before you do any optimisation work.

# Task 2

We will now look at an example of some code that I wrote a few years ago before I learned the above lesssons.

In this example we'll be performing the task called sigma clipping. The idea here is that you take a list of input values, compute the mean and standard deviation and then remove all values that fall outside some sigma range. The process is repeated until the list of items is stable (or zero).

In [ ]:
data = np.concatenate((np.linspace(9.5, 10.5, 31),
                    np.linspace(0, 20, 5)))
# it's important for our code that nan/inf values are rejected correctly so we include them in our test data
data[0] = np.nan
data[-1] = np.inf

In [ ]:
def sigmaclip(arr, lo=6, hi=6):
    """
    Perform sigma clipping on an array, ignoring non finite values.
    During each iteration return an array whose elements c obey:
    mean -std*lo < c < mean + std*hi
    where mean/std are the mean std of the input array.
    
    Continue until the members of c are stable.
    
    Parameters
    ----------
    arr : iterable
        An iterable array of numeric types.
    lo : float
        The negative clipping level.
    hi : float
        The positive clipping level.
        
    Returns
    -------
    mean : float
        The mean of the array, possibly nan
    std : float
        The std of the array, possibly nan
    """
    # Remove inf/nan values from our data
    clipped = [ a for a in arr if np.isfinite(a)]

    if len(clipped) < 1:
        return np.nan, np.nan

    std = np.std(clipped)
    mean = np.mean(clipped)
    elements = len(clipped)
    p_elements = elements+1
    while elements < p_elements:
        p_elements = elements
        clipped = [ a for a in clipped if a>mean-std*lo]
        clipped = [ a for a in clipped if a<mean+std*hi]
        elements = len(clipped)
        std = np.std(clipped)
        mean = np.mean(clipped)
    return clipped, mean, std

We want to compare this to the (nearly) equivalent function provided by scipy.
The scipy function does almost what we want so we write a small wrapper around it to get behaviour equivalent to the above.

NB: this was not part of scipy.stats at the time the above code was written!

In [ ]:
import scipy
import scipy.stats

In [ ]:
def scipy_sigmaclip(arr, lo, hi):
    """
    This is a wrapper around scipy.stats.sigma_clip that will give the same results as our own sigma_clip function.
    """
    # Remove inf/nan values from our data
    clipped = [ a for a in arr if np.isfinite(a)]
    
    # reading the help for scipy.stats.sigmaclip we see that the returned values are:
    # array, min value, max value
    # but we want:
    # array, mean, std
    clipped, _ ,_ = scipy.stats.sigmaclip(clipped, lo, hi)
    mean,std =  np.mean(clipped), np.std(clipped)
    return clipped, mean, std
    

As usual we want to make sure that our two functions agree before we do any profiling or optimisation work

In [ ]:
def compare_sigmas(data, lo, hi, func1, func2):
    ans1 = func1(data, lo, hi)
    ans2 = func2(data, lo, hi)
    print("The mean/std agree: ", ans1[1:] == ans2[1:])
    print("The returned arrays agree: ",np.all(ans1[0] == ans2[0]))

In [ ]:
compare_sigmas(data, 2, 2, sigmaclip, scipy_sigmaclip)

Now that they agree we can compare their run times:

In [ ]:
%timeit sigmaclip(data, 2, 2)

In [ ]:
%timeit scipy_sigmaclip(data, 2,2)

In [ ]:
# see where sigmaclip spends it's time by running a lineprofile on it
%lprun 

In [ ]:
# and now the same for our scipy wrapper function
%lprun 

What do you notice about the wrapper function and where it is spending time?

### Discussion: Which of the above two solutions would you use?
- which is the fastest?
- which is easier to develop?
- which is easier to maintain?

# Task 2: Optimisation practice

- Look over the line profiling for sigmaclip and identify some lines which have a non-negligible fraction of the total time spent on them.
- Identify a way to either speed up (or elliminate) these lines.

Suggestions:
- removing inf/nan values
- reconstructing the clipped array

In [ ]:
def sigmaclip2(arr, lo=6, hi=6):
    """
    As per sigmaclip2 but with some optimisation!
    """
    # Remove inf/nan values from our data
    clipped = [ a for a in arr if np.isfinite(a)]

    if len(clipped) < 1:
        return np.nan, np.nan

    std = np.std(clipped)
    mean = np.mean(clipped)
    elements = len(clipped)
    p_elements = elements+1
    while elements < p_elements:
        p_elements = elements
        clipped = [ a for a in clipped if a>mean-std*lo]
        clipped = [ a for a in clipped if a<mean+std*hi]
        elements = len(clipped)
        std = np.std(clipped)
        mean = np.mean(clipped)
    return clipped, mean, std

Remember to compare to the scipy_sigmaclip first to ensure that you have a working function

In [ ]:
compare_sigmas(data, 2, 2, sigmaclip2, scipy_sigmaclip)

In [ ]:
%timeit sigmaclip2(data,2,2)

In [ ]:
%lprun -f sigmaclip2 sigmaclip2(data,2,2)